# Big Data L2 - Reports with Apache Spark
## Чаплыгин Алексей 6131-010402D

#### Требования к окружению
Перед началом требуется выпаолнить следующий код в теминале, так как это позводт читать XML files через pyspark

```pyspark --packages com.databricks:spark-xml_2.12:0.13.0```

In [1]:
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, DateType

# SparkSession initialization

In [2]:
from pyspark.sql import SparkSession
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

spark_session = SparkSession\
    .builder\
    .getOrCreate()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
spark_session

In [4]:
sc = spark_session._sc

# Импорт данных и предобработка данных

In [5]:
!head /mnt/data/programming-languages.csv

name,wikipedia_url
A# .NET,https://en.wikipedia.org/wiki/A_Sharp_(.NET)
A# (Axiom),https://en.wikipedia.org/wiki/A_Sharp_(Axiom)
A-0 System,https://en.wikipedia.org/wiki/A-0_System
A+,https://en.wikipedia.org/wiki/A%2B_(programming_language)
A++,https://en.wikipedia.org/wiki/A%2B%2B
ABAP,https://en.wikipedia.org/wiki/ABAP
ABC,https://en.wikipedia.org/wiki/ABC_(programming_language)
ABC ALGOL,https://en.wikipedia.org/wiki/ABC_ALGOL
ABSET,https://en.wikipedia.org/wiki/ABSET


## Обявляем схему на основании .csv

In [6]:
prog_lang_schema = StructType([
    StructField("name", StringType(), False),
    StructField("wikipedia_url", StringType(), False),
])

In [7]:
df_prog_langs = spark_session.read.csv("/mnt/data/programming-languages.csv", schema=prog_lang_schema)

In [8]:
df_prog_langs.head()

Row(name='name', wikipedia_url='wikipedia_url')

## Формируем список языков программирвания

In [9]:
prog_langs_list = [str(row[0]) for row in df_prog_langs.select('name').collect()]

In [10]:
prog_langs_list[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

## Проверяем данные в XML файле

In [11]:
!head -n 3 /mnt/data/posts_sample.xml

���<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="4" PostTypeId="1" AcceptedAnswerId="7" CreationDate="2008-07-31T21:42:52.667" Score="630" ViewCount="42817" Body="&lt;p&gt;I want to use a track-bar to change a form's opacity.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;This is my code:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;decimal trans = trackBar1.Value / 5000;&#xA;this.Opacity = trans;&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;When I build the application, it gives the following error:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;Cannot implicitly convert type &lt;code&gt;'decimal'&lt;/code&gt; to &lt;code&gt;'double'&lt;/code&gt;&lt;/p&gt;&#xA;&lt;/blockquote&gt;&#xA;&#xA;&lt;p&gt;I tried using &lt;code&gt;trans&lt;/code&gt; and &lt;code&gt;double&lt;/code&gt; but then the control doesn't work. This code worked fine in a past VB.NET project.&lt;/p&gt;&#xA;" OwnerUserId="8" LastEditorUserId="3641067" LastEditorDisplayName="Rich B" LastEditDate="2019-07-19T01:39:54.173" 

In [12]:
df_posts_sample = spark_session.read.format("xml").options(rowTag="row").load('/mnt/data/posts_sample.xml')

In [13]:
df_posts_sample.head()

Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 16, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 7, 31, 21, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 1, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUse

# Сортируем языки программирования по популярности

Предположим что популярность основана на количествен упоминаний языка

In [14]:
def define_language(row):
    language_tag = None
    for lang in prog_langs_list:
        if '<' + lang.lower() + '>' in row._Tags.lower():
            language_tag = lang
            break
    if language_tag is None:
        return None
    return (row._Id, language_tag)


In [15]:
def check_date_range(row):
    left_border = datetime(year=2010, month=1, day=1)
    right_border = datetime(year=2020, month=1, day=1)
    created_date = row._CreationDate
    return created_date > left_border and created_date < right_border

```python
top_languages = df_posts_sample.rdd\  # Convert our DataFrame to RDD
    .filter(lambda row: row._Tags is not None and check_date_range(row))\  # It's pointless to use posts without tags, also let's check here our date boundaries
    .map(define_language)\  # Map our rows to (post_id, programming language)
    .filter(lambda row: row is not None)\  # In the previous step we might get "None" rows (due to not the all posts tagged with any programming language)
    .keyBy(lambda row: row[1])\  # Let the programming language be the key
    .aggregateByKey( 
        0,
        lambda acc, value: acc + 1,
        lambda acc1, acc2: acc1 + acc2,
    )\  # Calculates a number of posts for each language
    .sortBy(lambda row: row[1], ascending=False)\  # Sort rows by number of posts 
    .toDF()  # Backward convertation to DataFrame
```

In [16]:
top_languages = df_posts_sample.rdd\
    .filter(lambda row: row._Tags is not None and check_date_range(row))\
    .map(define_language)\
    .filter(lambda row: row is not None)\
    .keyBy(lambda row: row[1])\
    .aggregateByKey(
        0,
        lambda acc, value: acc + 1,
        lambda acc1, acc2: acc1 + acc2,
    )\
    .sortBy(lambda row: row[1], ascending=False)\
    .toDF()

## Подготавливаем и сохраняем DataFrame как .parquet

In [17]:
top_languages = top_languages.select(col('_1').alias('ProgrammingLanguage'), col('_2').alias('NumberOfPosts'))

In [18]:
top_languages.show()

+-------------------+-------------+
|ProgrammingLanguage|NumberOfPosts|
+-------------------+-------------+
|         JavaScript|         1799|
|               Java|         1518|
|                PHP|         1155|
|             Python|         1136|
|                  R|          313|
|                  C|          291|
|        Objective-C|          262|
|               Ruby|          178|
|               Bash|          103|
|              Scala|           92|
|             MATLAB|           88|
|         PowerShell|           68|
|               Perl|           51|
|             Delphi|           49|
|         TypeScript|           49|
|                 Go|           45|
|                AWK|           34|
|               Curl|           31|
|            Haskell|           31|
|                 F#|           22|
+-------------------+-------------+
only showing top 20 rows



In [19]:
top_languages.write.parquet("top_languages.parquet")

## Выводим 10 самых популярных языков отдельно и упаковываем в .parquet

In [20]:
parquetFile = spark_session.read.parquet("top_languages.parquet")

In [21]:
parquetFile.createOrReplaceTempView("parquetFile")
top_10 = spark_session.sql("SELECT ProgrammingLanguage FROM parquetFile ORDER BY NumberOfPosts DESC LIMIT 10")
top_10.show()

+-------------------+
|ProgrammingLanguage|
+-------------------+
|         JavaScript|
|               Java|
|                PHP|
|             Python|
|                  R|
|                  C|
|        Objective-C|
|               Ruby|
|               Bash|
|              Scala|
+-------------------+



In [22]:
top_10.write.parquet("top_10_langueages.parquet")